#               SUBWORD EMBEDDING WITH 1M DATA

- Model 1: basic Word2Vec algortihm. Inputs are complete words without splitting



- Model 2: Word2Vec algorithm where inputs are splitted based on some common suffixess and prefixess.



- Model 3: Word2Vec algorithm where inputs with more than 4 chcarcter, splitted based on compounds structure but also can be effective to detect suffixes (not prefix)



- Model 4: Combination of Model 2 and Model 3

# 1. IMPORTING THE LIBRARIES 

In [1]:
from gensim.models.word2vec import Word2Vec
import gensim
from gensim.models import Word2Vec
from gensim.models import FastText
from gensim.corpora import Dictionary
from statistics import mean 
from gensim.models.fasttext import FastText

# 2. READING THE FILES

## 2.1 Reading the Dataset

I downloaded text file with 3OOK sentences from the The Wortschatz Leipzig project (https://wortschatz.uni-leipzig.de/en/download). In this part I am reading and creating the dataset. 

In [2]:
#openning the text file 
text= open("eng-uk_web-public_2018_1M-sentences.txt")

#reading the file 
data = text.readlines()

In [3]:
#number of sentences in the data set is 
number_of_sentences = len(data)
print("Number of Sentences in the Dataset =",number_of_sentences)

Number of Sentences in the Dataset = 1000000


## 2.2 Reading the Word Similarity Dataset 1

Here, I have choosen wordsim353 as Word Similarity Dataset 1.(http://alfonseca.org/eng/research/wordsim353.html).

In [4]:
#openning the text file 
similarity_text_one = open('wordsim_similarity_goldstandard.txt' )


#reading the file 
similarity_dataset_one = similarity_text_one.readlines()

Now,  I haveto preprocess the Word Simlarity Dataset 1 to compare with trained dataset in the future. To do this.

First I will split the each line into words, make the value float and also transform the values into (-1,1). Because in the Word Similarity dataset , values are between 0-10, and in the Word"vec is between -1 to -1.

In [5]:

#for convenience, splitting the each line into tokens
similarity_one = []

for i in similarity_dataset_one:
    splitted= i.split()
    
    #tranlating the similarity value to float and normalizing by dividing 10 and adding-1
    splitted[2] = float(splitted[2])/5 -1
    
    similarity_one.append(splitted)
    
#example: 
print(similarity_one[1])

['tiger', 'tiger', 1.0]


## 2.3 Reading the Word Similarity Dataset 2 (with Rare Words)

Here I used SynEngl.txt that was provided in the Learnweb.

In [6]:
#openning the text file 
similarity_text_two = open('SynEngl.txt' )


#reading the file 
similarity_dataset_two = similarity_text_two.readlines() 

Now,  I have preprocess the word simlarity dataset 2 to compare with trained dataset in the future. To do this, first I will split the each line into words, translate the binary values "-,+" into "0,1". 

In [7]:

#for convenience, splitting the each line into words
similarity_two = []

for j in similarity_dataset_two:
    splitted= j.split()
    if splitted[2]=="-":
        splitted[2] = 0
    elif splitted[2]=="+":
        splitted[2]=1  
    similarity_two.append(splitted)
    
#example: 
print(similarity_two[104])

['allotments', 'parceling', 1]


# 4. TOKENIZATION: SPLITTING THE SENTENCES INTO WORDS

For the simplicity, i used gensim.utils.simple_preprocess function.

In [8]:
#creating a list of tokens of all sentences 
all_tokens = [] 

for i in range(0,1000000):
    tokens = gensim.utils.simple_preprocess(data[i])
    all_tokens.append(tokens)   

In [9]:
#example: 101 th sentence's token
print("101th sentence = \n", data[100])
print("101 th sentence's Token = \n", all_tokens[100])

101th sentence = 
 101	£160 per room per night to include Full English Breakfast.

101 th sentence's Token = 
 ['per', 'room', 'per', 'night', 'to', 'include', 'full', 'english', 'breakfast']


# 5. CREATING AN ACCURACY FUNCTION

To able to compare the accuracy of the Model 1, 2 and 3 , I need a accuracy calculator function. In this part,  I will create it.

In [10]:
# S: Word Similarity Dataset
# W: All traiend words
# M: Model

def accuracy(S,W,M):
    all_d = []
    for i in S:
        if i[0] in W and i[1] in W:
            model_similarity= M.wv.similarity(i[0], i[1])
            d = abs(abs(model_similarity) -i[2])
            all_d.append(d)
            
    print("Number Of Words That Accuracy Calculated = ",len(all_d),"\nAccuracy = ", 1-mean(all_d))
    

# 6. MODEL 1  (WITHOUT SPLITTING)

When I train the all tokens, I used Word2Vec algorithm.

## 6.1 Training the Model 1

Now, we will train the Model 1 with 32 workers.

In [11]:
#training the model 1 with 32 workers
model_one = Word2Vec(all_tokens,workers=32)

## 6.2 Example: Embedding Vector of a Word in Model 1

After traiend the 1M sentences, we obtained the word embedding for eact trained word.

In [12]:
#example: word embeddings of "supermarkets"
model_one["supermarket"]

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  


array([ 1.6022956 , -0.09645638,  0.41456208,  0.7070461 , -0.6661886 ,
        1.1985301 ,  0.5914592 , -0.716254  ,  0.36949208, -0.91004395,
        1.260426  ,  0.49890321, -0.14233308, -0.63430756,  0.2297844 ,
        0.10709617, -0.5457072 , -0.3388527 , -1.6424888 , -0.29356757,
        0.8893272 ,  0.1369259 ,  0.7374647 ,  0.12808241,  0.9817777 ,
       -1.0753342 , -0.9061321 ,  0.5377307 ,  0.27120274,  0.23889375,
        0.44479135,  0.77234054,  1.0183166 ,  0.05342788,  0.37945497,
        1.4214177 , -0.5205083 , -0.7313458 ,  0.74728495,  1.7161732 ,
       -0.31742644,  0.0505298 , -0.07494313,  0.47764987,  0.07112899,
       -0.2840139 , -0.15285781, -0.34173205,  0.51485544,  0.91036946,
       -0.72213537,  0.02989148,  1.5216497 ,  2.0125356 , -0.0567274 ,
        1.1050545 ,  0.0323332 ,  0.11865284,  1.3212317 ,  0.31516775,
        0.00324639,  0.3507848 ,  1.2431543 ,  0.02330622, -1.0592965 ,
        0.30768242, -0.54675114,  0.87938905,  0.86206174, -2.12

## 6.3  Example: Top 10 Similar to a Word 

In [13]:
# the most 10 similar words to discussion model 1
model_one.wv.most_similar('criminal', topn=10)  # get other similar words

[('prosecution', 0.701631486415863),
 ('alleged', 0.6908283233642578),
 ('unlawful', 0.6868874430656433),
 ('harassment', 0.6717197299003601),
 ('enforcement', 0.6681805849075317),
 ('discrimination', 0.6608335971832275),
 ('misconduct', 0.6506532430648804),
 ('sexual', 0.645127534866333),
 ('homicide', 0.6390079259872437),
 ('arrest', 0.6364690065383911)]

## 6.5 Example: Similarity Between Two Words

In [14]:
print("Similarity Super and Supermarkets = ", model_one.wv.similarity('super', 'supermarket'))

Similarity Super and Supermarkets =  0.2148754


# 7 MODEL 2 (WITH SPLITTING BASED ON PREFIX AND SUFFIX)

In this model, I created a function which splits the raw tokens (words). The created function takes into consideration the some most frequent suffixes and prefixes.

## 7.1 Splitting Function  (Words Based on Suffix and Prefix)

In [15]:
def splitting(sentence):
    liste = []
    for word in sentence:    
        n = len(word)
        
        if n>3:
            #SUFFIXES
            if word[n-2:n]=="ed":
                a = word[:-2]
                liste.append(word)
                liste.append(a)
                #liste.append("ed")
           
                
            elif word[n-3:n]=="ing":
                liste.append(word)
                a = word[:-3]
                liste.append(a)
                #liste.append("ing")
            
              
                
            elif word[-1] =="s":
                liste.append(word)
                a= word[:-1] 
                liste.append(a)
                #liste.append("s")
           
                
            elif word[n-4:n]=="able":
                liste.append(word)
                a = word[:-4]
                liste.append(a)
                #liste.append("able")
            
             
                
            elif word[n-5:n]=="tion":
                liste.append(word)
                a = word[:-5]
                liste.append(a)
                #liste.append("ation")
            
                
            elif word[n-4:n]=="hood":
                liste.append(word)
                a = word[:-4]
                liste.append(a)
                #liste.append("hood")
              
             
                
            elif word[n-2:n]=="er":
                liste.append(word)
                a = word[:-2]
                liste.append(a)
                #liste.append("er")
            
                
            elif word[n-2:n]=="ly":
                liste.append(word)
                a = word[:-2]
                liste.append(a)
                liste.append("ly")
               # sentence.remove(word)
                
            elif word[n-3:n]=="ist":
                liste.append(word)
                a = word[:-3]
                liste.append(a)
                #liste.append("ist")
           
                
                 
            elif word[n-2:n]=="es":
                liste.append(word)
                a = word[:-2]
                liste.append(a)
               # liste.append("es")
               
                
                 
            elif word[n-3:n]=="let":
                liste.append(word)
                a = word[:-3]
                liste.append(a)
                #liste.append("let")
            
            elif word[n-4:n]=="ness":
                liste.append(word)
                a = word[:-4]
                liste.append(a)
                #liste.append("let")
                
            
            elif word[n-4:n]=="ment":
                liste.append(word)
                a = word[:-4]
                liste.append(a)
                #liste.append("let")
     
            elif word[n-4:n]=="ment":
                liste.append(word)
                a = word[:-4]
                liste.append(a)
                #liste.append("less") 
                
            elif word[n-2:n]=="er":
                liste.append(word)
                a = word[:-2]
                liste.append(a)
                #liste.append("less") 
            
                
            elif word[n-4:n]=="ible":
                liste.append(word)
                a = word[:-4]
                liste.append(a)
                #liste.append("able")
                
                  
            elif word[n-3:n]=="ate":
                liste.append(word)
                a = word[:-3]
                liste.append(a)
                #liste.append("able")
                
                  
            elif word[n-2:n]=="fy":
                liste.append(word)
                a = word[:-2]
                liste.append(a)
                #liste.append("less")
                
             #PREFIXES      
            elif word[0:2]=="un":
                liste.append(word)
                a = word[2:n]
                liste.append(a)
                #liste.append("less")
                
            elif word[0:3]=="non":
                liste.append(word)
                a = word[3:n]
                liste.append(a)
                #liste.append("less")
            
            elif word[0:3]=="dis":
                liste.append(word)
                a = word[3:n]
                liste.append(a)
                #liste.append("less")
                
            elif word[0:4]=="anti":
                liste.append(word)
                a = word[4:n]
                liste.append(a)
                #liste.append("less")
                
                   
            elif word[0:3]=="sub":
                liste.append(word)
                a = word[3:n]
                liste.append(a)
                #liste.append("less")
                
            elif word[0:5]=="under":
                liste.append(word)
                a = word[5:n]
                liste.append(a)
                liste.append("under")
                
            elif word[0:5]=="super":
                liste.append(word)
                a = word[5:n]
                liste.append(a)
                liste.append("super")

            else:
                liste.append(word)          
              
    return liste

## 7.2 Splitting The Words

In [16]:
splitted_tokens = []
modified_dataset = all_tokens

for s in modified_dataset:
    splitted_tokens.append(splitting(s))

## 7.3 Training the Model 2 

In [17]:
model_two = Word2Vec(splitted_tokens,workers=32)

# 8. MODEL 3 (WITH SPLITTING BASED ON COMPOUNDS)

## 8.1 Creating Base Vocabulary Dataset

In [18]:
trained_words_model_one = list(model_one.wv.vocab) 
trained_words_model_two = list(model_two.wv.vocab)

vocabulary = set(trained_words_model_one + trained_words_model_two)

## 8.2 Splitting Function (Based on Compunds)

In [19]:
def splitting_two(sentence):
    liste=[]
    for word in sentence:
        n = len(word)
        #chossing the words with more than 4 character
        if n>6:
            for i in range(4,n):
                if word[0:i] in vocabulary:
                    liste.append(word[0:i])
                    if word[i:n] in trained_words_model_one:
                        liste.append(word[i:n]) 
                    
        liste.append(word)
    return liste
       
                

## 8.3 Splitting

In [20]:
splitted_tokens_two = []
modified_dataset_second = all_tokens

for s in modified_dataset_second:
    splitted_tokens_two.append(splitting_two(s))

In [21]:
splitted_tokens_two

[['wait',
  'ing',
  'waiting',
  'time',
  'tariff',
  'each',
  'period',
  'of',
  'one',
  'minute',
  'or',
  'part'],
 ['chri',
  'chris',
  'christ',
  'ian',
  'christi',
  'an',
  'christian',
  'cath',
  'catholic',
  'relabelled',
  'as',
  'chri',
  'chris',
  'christ',
  'ian',
  'christi',
  'an',
  'christian',
  'roman',
  'cath',
  'catholic',
  'for',
  'cons',
  'consist',
  'consistency',
  'with',
  'census',
  'labels'],
 ['per', 'night', 'per', 'person'],
 ['mile',
  'south',
  'of',
  'the',
  'town',
  'which',
  'is',
  'incl',
  'includ',
  'ed',
  'include',
  'included',
  'in',
  'the',
  'heritage',
  'walk',
  'vale',
  'trail'],
 ['mile',
  'you',
  'will',
  'see',
  'the',
  'entranc',
  'entrance',
  'to',
  'tŷ',
  'cerrig',
  'wood',
  'land',
  'woodland',
  'retreat',
  'retreats',
  'on',
  'your',
  'left'],
 ['sett',
  'ing',
  'setting',
  'bar',
  'with',
  'short',
  'temp',
  'temper',
  'tempera',
  'temperature',
  'probe'],
 ['home',
  

## 8.3 Training the Model 3

In [22]:
model_three = Word2Vec(splitted_tokens_two,workers=32)

# 9. COMPARISON BETWEEN MODEL 1, 2 , 3

## 9.1 Number of Trained Words

In [23]:
# all trained words in model 1 and model 2
trained_words_model_one = list(model_one.wv.vocab) #model 1
trained_words_model_two = list(model_two.wv.vocab) #model 2
trained_words_model_three = list(model_three.wv.vocab) #model 3

print("Number of Trained Words in Model 1 =",len(trained_words_model_one),"\n")
print("Number of Trained Words in Model 2 =",len(trained_words_model_two), "\n")
print("Number of Trained Words in Model 3 =",len(trained_words_model_three), "\n")

Number of Trained Words in Model 1 = 53239 

Number of Trained Words in Model 2 = 59510 

Number of Trained Words in Model 3 = 61392 



## 9.2 Accuracy Score on Word Similary Dataset 1

In [24]:
#calculation of the accuracy score for Word Similarity Dataset 1
print("Model 1: \n")
accuracy(similarity_one,trained_words_model_one,model_one)

Model 1: 

Number Of Words That Accuracy Calculated =  193 
Accuracy =  0.5803974439336537


In [25]:
print("Model 2 \n")
accuracy(similarity_one,trained_words_model_two,model_two)

Model 2 

Number Of Words That Accuracy Calculated =  195 
Accuracy =  0.5952802037190932


In [26]:
print("Model 3 \n")
accuracy(similarity_one,trained_words_model_three,model_three)

Model 3 

Number Of Words That Accuracy Calculated =  196 
Accuracy =  0.6187876678699894


## 9.3 Accuracy Score for Word Similary Dataset 2

In [27]:
#calculation of the accuracy score for Word Similarity Dataset 1
print("Model 1: \n")
accuracy(similarity_two,trained_words_model_one,model_one)

Model 1: 

Number Of Words That Accuracy Calculated =  756 
Accuracy =  0.519287184656908


In [28]:
print("Model 2 \n")
accuracy(similarity_two,trained_words_model_two,model_two)

Model 2 

Number Of Words That Accuracy Calculated =  785 
Accuracy =  0.519273881244052


In [29]:
print("Model 3 \n")
accuracy(similarity_two,trained_words_model_three,model_three)

Model 3 

Number Of Words That Accuracy Calculated =  785 
Accuracy =  0.5117550148492216
